In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

matplotlib.rc("axes", unicode_minus = False)

In [3]:
df1 = pd.read_csv("01 Oxidation.csv")
df2 = pd.read_csv("02 Photo_softbake.csv")
df3 = pd.read_csv("03 Photo_lithograpy.csv")
df4 = pd.read_csv("04 Etching.csv")
df5 = pd.read_csv("05 Ion_Implantation.csv")
df6 = pd.read_csv("06 Inspect.csv")

In [4]:
df = pd.merge(df1, df2, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df3, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df4, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df5, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df6, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])

In [5]:
df.dropna(subset="Thin F2", inplace=True) # 다른 변수들과의 연관성을 찾지못함, 한 행에 여러 열들 결측값 가짐
# pd.set_option('display.max_row', 200)
# pd.set_option('display.max_columns', 200)

In [6]:
df['Ox_Chamber'] = df['Ox_Chamber'].astype('str')
df['photo_soft_Chamber'] = df['photo_soft_Chamber'].astype('str')
df['lithography_Chamber'] = df['lithography_Chamber'].astype('str')
df['Etching_Chamber'] = df['Etching_Chamber'].astype('str')
df['Chamber_Num'] = df['Chamber_Num'].astype('str')
df['path'] = df['Ox_Chamber']+df['photo_soft_Chamber']+df['lithography_Chamber']+df['Etching_Chamber']+df['Chamber_Num']

In [7]:
df.drop(df[df['Oxid_time']<0].index, inplace=True) # 산화시간이 음수
df.drop(df[df['Target']==0].index, inplace=True) # target 값이 0
df.drop(columns = ["Wafer_map","Error_message"],inplace=True) # 웨이퍼맵, 에러메시지

In [8]:
# 시간 데이터 datetime 유형으로 변환
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y')

# 195이상 = 불량(1), 195미만 = 양품(0)
df.loc[df['Target'] >= 195, '불량_195이상'] =1
df.loc[df['Target'] < 195, '불량_195이상'] =0

In [9]:
df.loc[df['Thin F4']<0, 'Thin F4']=df['Thin F4'].median()
df.loc[df['Flux90s']<0, 'Flux90s']=df['Flux90s'].median()
df.loc[df['Flux160s']<0, 'Flux160s']=df['Flux160s'].median()
df.loc[df['Flux160s']<5, 'Flux160s']=df['Flux160s'].median()

In [10]:
df['Thin F1'].fillna(df['Thin F1'].median(), inplace=True)
df['Thin F3'].fillna(df['Thin F3'].median(), inplace=True)
df['Flux60s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux90s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux480s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux840s'].fillna(df['Flux90s'].median(), inplace=True)

### 불필요한 열 제거

In [11]:
# 불필요한 열 제거 
# df.drop(columns=["No_Die", "Lot_Num", "Wafer_Num"], inplace=True)
df.drop(columns=['Vapor','process','Wavelength'])
df.loc[df['Flux840s'] == 8.137500e+16, 'Flux840s'] = df['Flux840s'].mode()[0]
df.loc[df['Flux480s'] == 8.137500e+16, 'Flux480s'] = df['Flux480s'].mode()[0]

In [12]:
df_new = df[['Temp_OXid','ppm','Pressure','type','Oxid_time',
        'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'UV_type', 'Energy_Exposure',       
        'Temp_Etching', 'Source_Power',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       '불량_195이상', 'path']]

In [13]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1693 entries, 0 to 1703
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Temp_OXid          1693 non-null   float64
 1   ppm                1693 non-null   float64
 2   Pressure           1693 non-null   float64
 3   type               1693 non-null   object 
 4   Oxid_time          1693 non-null   int64  
 5   N2_HMDS            1693 non-null   float64
 6   pressure_HMDS      1693 non-null   float64
 7   temp_HMDS          1693 non-null   float64
 8   temp_HMDS_bake     1693 non-null   float64
 9   time_HMDS_bake     1693 non-null   float64
 10  spin1              1693 non-null   float64
 11  spin2              1693 non-null   float64
 12  spin3              1693 non-null   float64
 13  photoresist_bake   1693 non-null   float64
 14  temp_softbake      1693 non-null   float64
 15  time_softbake      1693 non-null   float64
 16  UV_type            1693 non-n

In [13]:
# 데이터 구성:Series, DataFrame
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
# scaling
from sklearn.preprocessing import StandardScaler
# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 로지스틱 회귀
from statsmodels.api import Logit
# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, f1_score 
from sklearn.metrics import confusion_matrix, classification_report

In [24]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

df_raw_x = df_new.drop("불량_195이상", axis = 1)
df_raw_y = df_new["불량_195이상"] 

# MinMaxScaler 객체 생성
numeric_columns = df_raw_x.select_dtypes(include=['number']).columns
scaler = MinMaxScaler()

df_raw_x[numeric_columns] = scaler.fit_transform(df_raw_x[numeric_columns])

df_raw_x = pd.get_dummies(df_raw_x, columns=df_raw_x.select_dtypes(include=['object']).columns)
df_raw_x = df_raw_x.astype({column: int for column in df_raw_x.select_dtypes(include=['bool']).columns})

# df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
#     df_raw_x, df_raw_y, test_size = 0.3, random_state = 1234, stratify=df_raw_y) 

In [44]:
# train_test_split(데이터, test_size = test 데이터 비율, random_state: 랜덤)
df_train, df_test = train_test_split(df_scaled, # 데이터
                                     test_size = 0.3, # test 데이터의 비율
                                     random_state = 1234)  # random state

print("train data size : {}".format(df_train.shape))
print("test data size : {}".format(df_test.shape))

train data size : (1192, 26)
test data size : (512, 26)


In [48]:
from statsmodels.formula.api import logit

# 로지스틱 회귀 모델 생성
log_model = logit("불량_195이상 ~ Temp_OXid + ppm + Pressure + C(type) + Oxid_time + N2_HMDS + \
pressure_HMDS + temp_HMDS + temp_HMDS_bake + time_HMDS_bake + spin1 + spin2 + spin3 + \
photoresist_bake + temp_softbake + time_softbake + C(UV_type) + Energy_Exposure + Temp_Etching \
+ Source_Power + input_Energy + Temp_implantation + Furance_Temp + RTA_Temp", df_train)

# 모델 학습 (BFGS 최적화 알고리즘 사용)
log_result = log_model.fit(method='bfgs', maxiter=100)

# 모델 요약
print(log_result.summary())

Optimization terminated successfully.
         Current function value: 0.228165
         Iterations: 67
         Function evaluations: 68
         Gradient evaluations: 68
                           Logit Regression Results                           
Dep. Variable:               불량_195이상   No. Observations:                 1178
Model:                          Logit   Df Residuals:                     1152
Method:                           MLE   Df Model:                           25
Date:                Tue, 19 Mar 2024   Pseudo R-squ.:                  0.1341
Time:                        14:05:16   Log-Likelihood:                -268.78
converged:                       True   LL-Null:                       -310.40
Covariance Type:            nonrobust   LLR p-value:                 3.514e-08
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -2.

In [69]:
from statsmodels.formula.api import logit

# 로지스틱 회귀 모델 생성
log_model = logit("불량_195이상 ~ Temp_OXid + Pressure + C(type)   + \
 spin1  + photoresist_bake + C(UV_type)  ", df_train)

# 모델 학습 (BFGS 최적화 알고리즘 사용)
log_result = log_model.fit(method='bfgs', maxiter=100)

# 모델 요약
print(log_result.summary())

Optimization terminated successfully.
         Current function value: 0.234066
         Iterations: 49
         Function evaluations: 50
         Gradient evaluations: 50
                           Logit Regression Results                           
Dep. Variable:               불량_195이상   No. Observations:                 1178
Model:                          Logit   Df Residuals:                     1170
Method:                           MLE   Df Model:                            7
Date:                Tue, 19 Mar 2024   Pseudo R-squ.:                  0.1117
Time:                        14:08:56   Log-Likelihood:                -275.73
converged:                       True   LL-Null:                       -310.40
Covariance Type:            nonrobust   LLR p-value:                 2.011e-12
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -2.839

In [71]:
# train 데이터 예측
y_pred_train = log_result.predict(df_train)
# 0과 1의 값을 가진 class로 변환
y_pred_train_class = (y_pred_train > 0.5).astype(int)  # 0.5 : “1/0” 판정 임계값(1 발생 확률) 변경 가능 
print("Train 예측 결과 \n", y_pred_train_class.head(), "\n")
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["불량_195이상"],y_pred_train_class)),"\n")

# test 데이터 예측
y_pred_test = log_result.predict(df_test)
# 0과 1의 값을 가진 class로 변환
y_pred_test_class = (y_pred_test > 0.5).astype(int)
print("Test 예측 결과 \n", y_pred_test_class.head(),"\n")
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["불량_195이상"],y_pred_test_class)),"\n")

Train 예측 결과 
 1153    0
27      0
1017    0
781     0
89      0
dtype: int64 

Confusion Matrix: 
[[1105    0]
 [  83    4]] 

Test 예측 결과 
 216     0
1115    0
188     0
1144    0
803     0
dtype: int64 

Confusion Matrix: 
[[478   0]
 [ 31   3]] 



In [72]:
# 실제 train 데이터와 예측 결과 비교
print("Train 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_train["불량_195이상"], y_pred_train_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["불량_195이상"],y_pred_train_class)),"\n")
print(classification_report(df_train["불량_195이상"], y_pred_train_class, digits=3))

# 실제 train 데이터와 예측 결과 비교
print("Test 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_test["불량_195이상"], y_pred_test_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["불량_195이상"],y_pred_test_class)),"\n")
print(classification_report(df_test["불량_195이상"], y_pred_test_class, digits=3))

Train 예측/분류 결과
Accuracy: 0.930

Confusion Matrix: 
[[1105    0]
 [  83    4]] 

              precision    recall  f1-score   support

           0      0.930     1.000     0.964      1105
           1      1.000     0.046     0.088        87

    accuracy                          0.930      1192
   macro avg      0.965     0.523     0.526      1192
weighted avg      0.935     0.930     0.900      1192

Test 예측/분류 결과
Accuracy: 0.939

Confusion Matrix: 
[[478   0]
 [ 31   3]] 

              precision    recall  f1-score   support

           0      0.939     1.000     0.969       478
           1      1.000     0.088     0.162        34

    accuracy                          0.939       512
   macro avg      0.970     0.544     0.565       512
weighted avg      0.943     0.939     0.915       512



In [26]:
# train_test_split(데이터, test_size = test 데이터 비율, random_state: 랜덤)
X_train, X_test , y_train, y_test = train_test_split(df_raw_x, df_raw_y, # 데이터
                                     test_size = 0.3, # test 데이터의 비율
                                     random_state = 1234)  # random state

In [27]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

y_pred = logistic_model.predict(X_test)
print("Confusion matrix: \n{}".format(confusion_matrix(y_test, y_pred)))
print(classification_report(y_test, y_pred))

Confusion matrix: 
[[464   0]
 [ 43   1]]
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96       464
         1.0       1.00      0.02      0.04        44

    accuracy                           0.92       508
   macro avg       0.96      0.51      0.50       508
weighted avg       0.92      0.92      0.88       508



In [31]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

# SMOTE를 적용하여 불균형 데이터를 조정
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 로지스틱 회귀모델 학습
logistic_model = LogisticRegression()
logistic_model.fit(X_resampled, y_resampled)

# 모델 성능 평가
y_pred = logistic_model.predict(X_test)
print("Confusion matrix: \n{}".format(confusion_matrix(y_test, y_pred)))
print(classification_report(y_test, y_pred))

Confusion matrix: 
[[420  44]
 [ 32  12]]
              precision    recall  f1-score   support

         0.0       0.93      0.91      0.92       464
         1.0       0.21      0.27      0.24        44

    accuracy                           0.85       508
   macro avg       0.57      0.59      0.58       508
weighted avg       0.87      0.85      0.86       508

